# はじめに
- 「TensorFlowで始めるDeepLearning実践入門」を読みながらメモをしたnotebook

# 計算グラフとDefine and Run
- Define and Run : 宣言と実行を分ける。計算グラフを定義し、その後に値の代入と実行が行われる
    - TensorFlow, Theano
- Define by Run : 宣言と実行が同時に行われる。pythonで何も考えないときはこっちの考え方
    - Chainer, Torch
- TensorFlowとChainerではプログラミングの思想が違う。一見するとChainerが書きやすいが、抽象度が増して最終的に効率が良いのはTensorFlowだったりするのかな？

In [1]:
import tensorflow as tf

/home/yoichi/.pyenv/versions/anaconda2-4.3.0/envs/choco3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
a = tf.constant(3, name='const1') # 定数を出力するオペレーション
b = tf.Variable(0, name='val1') # 変数を出力するオペレーション

# a+bの計算グラフを作る
add = tf.add(a, b)

# 変数bに足した結果をアサイン。変数に値を割り当てる。
assign = tf.assign(b, add)

c = tf.placeholder(tf.int32, name='input') # 実行するまで値が決まらないもの。型だけ宣言。
# 実行時には、辞書型で値を入れる

# アサインした結果とcを掛ける
mul = tf.multiply(assign, c)

# 変数の初期化のためのオペレーションを作成
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init) # 初期化を実行
    for i in range(3):
        print(sess.run(mul, feed_dict={c:3}))
        # feed_dict : placeholderに値を渡すための引数


9
18
27


- tf.Session : 構築した計算グラフを実行するための概念
    - sessionの中で計算グラフを引数にしてrunすると、計算グラフが実行される

# MLPでMNIST

## データ取得

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# MNISTデータのオブジェクトを呼び出す
mnist = input_data.read_data_sets("data/", one_hot=True)

# 訓練データの取得
# ## ミニバッチを指定して、バッチ分づつデータを取得するオペレーションを作成
train_images, train_labels = mnist.train.next_batch(50)

# テスト用データの取得
test_images = mnist.test.images
test_labels = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(50, 784)
(50, 10)
(10000, 784)
(10000, 10)


## モデル定義

In [5]:
# 入力層
x = tf.placeholder(tf.float32, [None, 784])
# ** 入力データは固定のデータではないので、placeholderで定義
# ** バッチ分入力するがサイズは未指定としてNoneを入力

# 中間層
w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name="w1")
    # truncated_normal : 正規分布で初期値を指定する
b_1 = tf.Variable(tf.zeros([64]), name="b1")
h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)
    # 内積(FC層)してreluを掛ける

# 出力層
w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name="w2")
b_2 = tf.Variable(tf.zeros([10]), name="b2")
out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

- tf.truncated_normal : 切断正規分布を生成する。正規分布のminとmaxをある値で切った分布。stddevの2倍が設定されるらしい

## 誤差関数の定義

In [6]:
y = tf.placeholder(tf.float32, [None, 10]) # 正解データを入力する箱を用意
# 誤差関数の定義
loss = tf.reduce_mean(tf.square(y - out))

# 訓練のオペレーション定義
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # SGDをoptimizerとして最適化する。学習率は0.5

- reduce_mean:テンソルの平均を計算するメソッド。axis引数で平均をとる軸を指定できる。指定しない場合、全要素の平均を計算。numpy.meanと同じ動き。
- minimize:最適化の命令。内部ではcompute_gradients()とapply_gradients()を続けて実行。分散処理をする場合には、分けて実行する必要がある。が、通常は、minimizeを呼べば良い。


## 評価

In [7]:
correct = tf.equal(tf.argmax(out,1), tf.argmax(y,1)) # argmaxの第2引数はaxis
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## 計算グラフの実行

In [8]:
# 初期化
init = tf.global_variables_initializer()

# 実行
with tf.Session() as sess:
    sess.run(init)
    
    #テストデータをロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels
    
    for i in range(1000):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images, y:train_labels})
        
        if step % 10 == 0:
            acc_val = sess.run(accuracy, feed_dict={x:test_images, y:test_labels})
            print('Step %d: accuracy = %.2f'%(step, acc_val))

Step 10: accuracy = 0.13
Step 20: accuracy = 0.15
Step 30: accuracy = 0.17
Step 40: accuracy = 0.20
Step 50: accuracy = 0.23
Step 60: accuracy = 0.29
Step 70: accuracy = 0.36
Step 80: accuracy = 0.40
Step 90: accuracy = 0.44
Step 100: accuracy = 0.46
Step 110: accuracy = 0.45
Step 120: accuracy = 0.47
Step 130: accuracy = 0.47
Step 140: accuracy = 0.49
Step 150: accuracy = 0.53
Step 160: accuracy = 0.54
Step 170: accuracy = 0.57
Step 180: accuracy = 0.60
Step 190: accuracy = 0.61
Step 200: accuracy = 0.64
Step 210: accuracy = 0.65
Step 220: accuracy = 0.66
Step 230: accuracy = 0.68
Step 240: accuracy = 0.68
Step 250: accuracy = 0.70
Step 260: accuracy = 0.71
Step 270: accuracy = 0.72
Step 280: accuracy = 0.73
Step 290: accuracy = 0.73
Step 300: accuracy = 0.74
Step 310: accuracy = 0.74
Step 320: accuracy = 0.76
Step 330: accuracy = 0.76
Step 340: accuracy = 0.77
Step 350: accuracy = 0.78
Step 360: accuracy = 0.77
Step 370: accuracy = 0.78
Step 380: accuracy = 0.80
Step 390: accuracy = 